In [1]:
from libraries import lib
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

In [16]:
N = 2
J = 10
Gamma = 1

In [17]:
h = lib.TFIM_hamiltonian(N, J, Gamma)
eig = h.eigenstates()
print(eig[0][0])
print(eig[1][0])

-20.09975124224179
Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.70622893]
 [0.03522361]
 [0.03522361]
 [0.70622893]]


In [39]:
def amp_phase(nn_output):
    return nn_output[:, 0] * torch.exp(1.j * 2 * np.pi * nn_output[:, 1])
input = lib.generate_input_torch(N)
model = nn.Sequential(
    nn.Linear(N, 5 * N),
    nn.Sigmoid(),
    nn.Linear(5 * N, 2),
    nn.Sigmoid()
)

In [44]:
vars = (N, J, Gamma)
loss_fn = lib.TFIM_expectation_from_torch
epochs = []
loss_data = []
optimizer = torch.optim.SGD(model.parameters(), lr = 5)
for epoch in range(1000):
    y_pred = model(input)
    loss = loss_fn(y_pred, vars, amp_phase)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        loss_data.append(loss.item())
        epochs.append(epoch)

# find groud state
gs = lib.model_to_ground_state(model, input, amp_phase)

In [45]:
print(loss_data[-1])
gs

-20.09975242614746


Quantum object: dims=[[4], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[-0.68480456+0.17270432j]
 [-0.03415452+0.00861351j]
 [-0.03415445+0.0086135j ]
 [-0.684771  +0.17269412j]]